# SQL

В быстроизменяющееся время огрнаичений и изоляций, компания купила крупный сервис для чтения книг по подписке. 

**Цель:** 

Проанализировать базу данных такого сервиса. Ведь эти данные помогут сформулировать ценностное предложение для
нового продукта.

**Задача:**

Ответить на следующие вопросы:

* Сколько книг вышло после 1 января 2000 года?
* Для каждой книги определим количество обзоров и среднюю оценку;
* Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (таким образом исключии из анализа брошюры);
* Определим автора с самой высокой средней оценкой книг (будем учитывать только книги с 50 и более оценками);
* Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.



# Импорт библиотек, определение функций

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#функцие будем передавать SQL запрос, она выводить ответ
def sql_request(request):
    query=request
    temp = pd.io.sql.read_sql(query, con = engine)
    return temp

In [3]:
#функцие будем передавать датасет, в цикле будем перебирать столбцы
def dataset_info(dataset):
    count = 0
    dataset.columns = dataset.columns.str.lower().str.replace(' ','_') #тут приведем название столбцов к snake_case
    for column in dataset.columns:
        count += 1
        print('Название столбца:     ',column)                          #покажет как называется столбец
        print('Длина столбца:        ',len(dataset[column]))            #отобразит длину столбца
        print('Количество пропусков: ',dataset[column].isna().sum())    #отобразит количество пропусков
        print('Тип данных в столбце: ',dataset[column].dtype)           #покажет тип данных в столбце
        print('Пример значений:      ')                                 
        print(dataset[column].sample(5))                                 #отобразит уникальные значения
        print('----------------')
    print('Всего столбцов в датасете:', count)                          #подсчитаем количество столбцов в датасете
    print('Количество дубликатов: ',dataset.duplicated().sum())         #подсчитает количество полных строк-дубликатов

## Знакомство с данными

Наши данные - информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. А именно 5 таблиц: `books`, `authors`, `publishers`, `ratings`, `reviews`. Познакомимся с ними

### Таблица books

In [4]:
#загрузим данные в переменную и выведем 5 строк таблицы
query = ''' SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица books**

Содержит данные о книгах:

* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

In [5]:
#передадим функцие таблицу для анализа
dataset_info(books)

Название столбца:      book_id
Длина столбца:         1000
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
462    463
987    988
56      57
375    376
692    693
Name: book_id, dtype: int64
----------------
Название столбца:      author_id
Длина столбца:         1000
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
516    270
277    476
199    551
950    540
756    127
Name: author_id, dtype: int64
----------------
Название столбца:      title
Длина столбца:         1000
Количество пропусков:  0
Тип данных в столбце:  object
Пример значений:      
949                                  Uglies (Uglies  #1)
233                     Fables  Vol. 1: Legends in Exile
114        Bloody Bones (Anita Blake  Vampire Hunter #5)
681    The Color of Water: A Black Man's Tribute to H...
916                        The Yiddish Policemen's Union
Name: title, dtype: object
----------------
Название столбца:      num_pages
Длина столбца:         1000

В таблице 6 столбцов и 1000 строк, пропусков не обнаружено, дубликатов тоже. В `publication_date` следовало бы преобразовать тип данных.

### Таблица authors

In [6]:
#загрузим данные в переменную и выведем 5 строк таблицы
query = ''' SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица authors**

Содержит данные об авторах:

* `author_id` — идентификатор автора;
* `author` — имя автора.

In [7]:
#передадим функцие таблицу для анализа
dataset_info(authors)

Название столбца:      author_id
Длина столбца:         636
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
107    108
177    178
187    188
465    466
506    507
Name: author_id, dtype: int64
----------------
Название столбца:      author
Длина столбца:         636
Количество пропусков:  0
Тип данных в столбце:  object
Пример значений:      
351              Kurt Vonnegut Jr.
265                     Jane Green
289                   Joanne Fluke
323       Judi Barrett/Ron Barrett
328    Julia Child/Alex Prud'Homme
Name: author, dtype: object
----------------
Всего столбцов в датасете: 2
Количество дубликатов:  0


В таблице 2 столбца и 636 строк, пропусков не обнаружено, дубликатов тоже, тип данных корректный.

### Таблица publishers

In [8]:
#загрузим данные в переменную и выведем 5 строк таблицы
query = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица publishers**

Содержит данные об издательствах:

* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

In [9]:
#передадим функцие таблицу для анализа
dataset_info(publishers)

Название столбца:      publisher_id
Длина столбца:         340
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
71      72
222    223
120    121
147    148
224    225
Name: publisher_id, dtype: int64
----------------
Название столбца:      publisher
Длина столбца:         340
Количество пропусков:  0
Тип данных в столбце:  object
Пример значений:      
64                         Corgi
332    William Morrow Paperbacks
286            TIme Warner Books
164                   Kensington
300             Turtleback Books
Name: publisher, dtype: object
----------------
Всего столбцов в датасете: 2
Количество дубликатов:  0


В таблице 2 столбца и 340 строк, пропусков не обнаружено, дубликатов тоже, типы данных правильные.

### Таблица ratings

In [10]:
#загрузим данные в переменную и выведем 5 строк таблицы
query = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица ratings**

Содержит данные о пользовательских оценках книг:

* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

In [11]:
#передадим функцие таблицу для анализа
dataset_info(ratings)

Название столбца:      rating_id
Длина столбца:         6456
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
713      714
4533    4534
2905    2906
2733    2734
5565    5566
Name: rating_id, dtype: int64
----------------
Название столбца:      book_id
Длина столбца:         6456
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
4979    766
471      75
3701    607
2947    461
4890    750
Name: book_id, dtype: int64
----------------
Название столбца:      username
Длина столбца:         6456
Количество пропусков:  0
Тип данных в столбце:  object
Пример значений:      
3216    abbottjames
5678    lesliegibbs
5198          ehall
3612         ryan01
3091       robert00
Name: username, dtype: object
----------------
Название столбца:      rating
Длина столбца:         6456
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
3813    5
4166    3
1864    4
3690    5
4851    2
Name: rating, dtype: int64
------------

В таблице 4 столбца и 6456 строк, пропусков не обнаружено, дубликатов тоже, тип данных корректный.

### Таблица reviews

In [12]:
#загрузим данные в переменную и выведем 5 строк таблицы
query = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица reviews**

Содержит данные о пользовательских обзорах на книги:

* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, написавшего обзор;
* `text` — текст обзора.

In [13]:
#передадим функцие таблицу для анализа
dataset_info(reviews)

Название столбца:      review_id
Длина столбца:         2793
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
1241    1241
970      971
282      281
1369    1369
195      195
Name: review_id, dtype: int64
----------------
Название столбца:      book_id
Длина столбца:         2793
Количество пропусков:  0
Тип данных в столбце:  int64
Пример значений:      
1041    382
2059    740
2428    869
537     197
1612    584
Name: book_id, dtype: int64
----------------
Название столбца:      username
Длина столбца:         2793
Количество пропусков:  0
Тип данных в столбце:  object
Пример значений:      
22      williamsangela
806         ryanfranco
1406      valenciaanne
2181             jhill
2599       conradanita
Name: username, dtype: object
----------------
Название столбца:      text
Длина столбца:         2793
Количество пропусков:  0
Тип данных в столбце:  object
Пример значений:      
2715    Receive Mrs prove I field south also. Approach...
992     Beat tabl

В таблице 4 столбца и 2793 строк, пропусков не обнаружено, дубликатов тоже, тип данных корректный.

## Задачи

### Подсчет книг после 1 января 2000 года

In [14]:
sql_request(
    ''' 
SELECT COUNT(book_id) books_count
FROM books
WHERE publication_date > '2000-01-01' 
        '''
)

,books_count
0,819


После 1 января 2000 года вышло **819** книг.

### Определение количества обзоров и средней оценки для каждой книги

In [15]:
sql_request(
    ''' SELECT b.book_id as id, title,
               COUNT(DISTINCT(review_id)) review_count,
               ROUND(avg(rating),2) avg_ratio
        FROM books b
        LEFT JOIN ratings r ON b.book_id=r.book_id
        LEFT JOIN reviews rw ON b.book_id=rw.book_id 
        GROUP BY id
        '''
)

,id,title,review_count,avg_ratio
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


Для каждой книги был рассчитан средний рейтинг и количество обзоров на эту книгу. Таких книг получилось 1000 штук.

### Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [16]:
sql_request(
    ''' 
SELECT publisher, COUNT(DISTINCT(book_id)) book_count
FROM books b
LEFT JOIN publishers p ON b.publisher_id=p.publisher_id
WHERE num_pages > 50 
GROUP BY publisher
ORDER BY book_count DESC
LIMIT 1
        '''
)

,publisher,book_count
0,Penguin Books,42


Издательство **Penguin Books** выпустило больше всего книг, а именно **42**. В расчет брались книги, в которых больше 50 страниц.

### Определение автора с самой высокой средней оценкой книг

In [27]:
sql_request(
    ''' 
WITH ratio AS (
SELECT b.book_id AS id,  b.title, b.author_id, COUNT(rating) AS ratings_cnt
FROM books b
LEFT JOIN ratings r on b.book_id = r.book_id
GROUP BY id
HAVING COUNT(rating) > 50)

SELECT author, AVG(rating) as average_rating
FROM ratio
LEFT JOIN authors a on ratio.author_id = a.author_id
LEFT JOIN ratings r on ratio.id = r.book_id
GROUP BY author
ORDER BY average_rating DESC
LIMIT 3
        '''
)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914


Самую высокую среднюю оценку (среди книг, у которых более 50 оценок) имеет автор - **J.K. Rowling/Mary GrandPré**

### Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок

In [30]:
sql_request(
    ''' 
WITH top_ratio_user AS (
SELECT username, COUNT(rating) AS ratings_cnt
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50),

review_value AS (
SELECT top_ratio_user.username AS user, COUNT(review_id) as review_cnt
FROM top_ratio_user
LEFT JOIN reviews  r on top_ratio_user.username = r.username
GROUP BY top_ratio_user.username)

SELECT ROUND(AVG(review_cnt), 2)
FROM review_value
        '''
)

,round
0,24.33


Среднее количество обзоров от пользователей (которые поставили больше 50 оценок) - **24,33**

## Вывод

Данные в отличном состоянии, пропусков нет, дубликаты отсутствуют. Лишь в одной таблице и одном столбце необходимо привести тип данных к корректному виду.

Ответив на поставленные вопросы, можно сфомировать ценностное предложение:

В нашей базе книг содеражатся порядка 819 книг, вышедших в новом тысячелетии. От известных издательств и авторов, таких как J.K. Rowling/Mary GrandPré. Для каждой книги, найдете обзор и среднюю оценку от наших книголюбов, ведь они самые читающие - на одного приходится по 24 отзыва, которые помогут определиться с выбором.